## 1.导入包

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import re
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

np.set_printoptions(threshold=np.inf)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
"""
网格搜索(Grid Search)和Optuna，找到模型的最佳超参数组合
网格搜索适用于超参数空间较小、离散且较少的情况，而Optuna适用于超参数空间较大、连续或离散且较多的情况
下面要做的事情：
1.换新的面心值标签，现在数据过拟合，训练集下降但是测试集上升或者波动
2.考虑正则化或者假如droput层来防止过拟合
3.考虑数据预处理中采用数据标准化，让数据均匀分布
4.用不用考虑损失函数，学习率,epoch,adam优化器及其四个参数，的修改，模型用不用再添加几层让模型变复杂些（batch-size越大，训练越快，不影响准确率）
5.早停法（Early Stopping）：在训练过程中监控验证集上的性能，一旦性能停止改善，在一定epoch后停止训练，并保存模型，以防止过拟合。可以参照外国那案例
6.数据集的比例，不一定4：1，也可以95：5，当数据集足够大时，这样可以增加训练集数量
"""

'\n网格搜索(Grid Search)和Optuna，找到模型的最佳超参数组合\n网格搜索适用于超参数空间较小、离散且较少的情况，而Optuna适用于超参数空间较大、连续或离散且较多的情况\n下面要做的事情：\n1.换新的面心值标签，现在数据过拟合，训练集下降但是测试集上升或者波动\n2.考虑正则化或者假如droput层来防止过拟合\n3.考虑数据预处理中采用数据标准化，让数据均匀分布\n4.用不用考虑损失函数，学习率,epoch,adam优化器及其四个参数，的修改，模型用不用再添加几层让模型变复杂些（batch-size越大，训练越快，不影响准确率）\n5.早停法（Early Stopping）：在训练过程中监控验证集上的性能，一旦性能停止改善，在一定epoch后停止训练，并保存模型，以防止过拟合。可以参照外国那案例\n6.数据集的比例，不一定4：1，也可以95：5，当数据集足够大时，这样可以增加训练集数量\n'

## 2.加载数据

"""
1.功能：
通过加载data和label文件，然后继续训练和预测。
定义了一个6层卷积神经网络模型。每个卷积层后面跟着一个 ReLU 激活函数。第七层只有卷积，没有relu。
输入数据n*64*64*2,这里的一个样本64*64可以看成一个图片格式（在此次任务中是速度，两者类似）
输出是n*64*64*4
"""
"""txt保存为numpy格式发现可以减少存储大小，约缩小成1/4
5.9G	./all_data.npy
12G	./all_label.npy
27G	./data_64x64x2.txt
53G	./label_a_2x64x65x2.txt
"""

In [3]:
# 直接加载npy文件为numpy格式
all_data = np.load('./data/all_data.npy')
# #直接加载npy文件为numpy格式
all_label = np.load('./data/all_centerFace_label.npy')

all_data = torch.tensor(all_data).float()
all_label = torch.tensor(all_label).float()


In [4]:

class Net(nn.Module):
    def __init__(self, num_output_channels, dropout, layers, hidden_units):
        super(Net, self).__init__()

        self.layers = nn.ModuleList()  # 用于存储每个层的列表

        # 添加卷积层、激活函数、dropout层
        for i in range(layers):
            if i == 0:
                self.layers.append(nn.Conv2d(2, hidden_units, kernel_size=3, stride=1, padding=1))
            else:
                self.layers.append(nn.Conv2d(hidden_units, hidden_units, kernel_size=3, stride=1, padding=1))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.Dropout(p=dropout))

        # 添加最后一层卷积层
        self.conv_final = nn.Conv2d(hidden_units, num_output_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # 运行每个层的forward方法
        for layer in self.layers:
            x = layer(x)

        x = self.conv_final(x)

        return x

## 3.加载模型

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 加载模型
model = torch.load('./model/model_300_optuna_trainloss_326_layer16_03.pth', map_location=device)# from torchsummary import summary
# # 将模型移动到适当的设备
model = model.to(device)


## 4.预测

In [22]:
# 输入数据进行预测
input_data =all_data[-10]  # 你的输入数据
print("all_data[0].shape:",input_data.shape)

#调整输入input的维度顺序,作为E，用于下面change_Label_to_a中(E-A)/(B-A)得到a值
matrix_64 = input_data.cpu()
matrix_64 = matrix_64.permute(1,2,0)  
print(matrix_64.shape)


# 转换为四维
input_data = input_data.unsqueeze(0)#用实际数据，数据格式为(1,2, 64, 64)，不能为2x64x64
# input_data = torch.randn(1,2, 64, 64)
print("input_data.shape:",input_data.shape)#input_data.shape: torch.Size([1, 2, 64, 64])
input_tensor = input_data.to(device)

# 设置模型为评估模式,不启用dropout
model.eval()
with torch.no_grad():
    output = model(input_tensor)##如果报错的话需要把网络的设计加上，里面涉及model(input)

# 打印预测结果
# print(output)

all_data[0].shape: torch.Size([2, 64, 64])
torch.Size([64, 64, 2])
input_data.shape: torch.Size([1, 2, 64, 64])


In [23]:

# total_memory = torch.cuda.get_device_properties(device).total_memory
# allocated_memory = torch.cuda.memory_allocated(device)
# cached_memory = torch.cuda.memory_reserved(device)
# print("总显存：", total_memory)
# print("已分配显存：", allocated_memory)
# print("缓存显存：", cached_memory)


In [24]:
print(all_label[-10].cpu().numpy())

[[[ 2.76224995e+00  2.79293871e+00  2.83389115e+00  2.88520002e+00
    2.94344497e+00  3.00627136e+00  3.07503629e+00  3.15504241e+00
    3.25364137e+00  3.37030196e+00  3.48562074e+00  3.59064317e+00
    3.71066070e+00  3.85190868e+00  3.95558882e+00  3.96324992e+00
    3.90299821e+00  3.83000183e+00  3.76928258e+00  3.72359371e+00
    3.68938136e+00  3.66392875e+00  3.64676881e+00  3.63888931e+00
    3.64156055e+00  3.65552616e+00  3.68060565e+00  3.71558428e+00
    3.75827503e+00  3.80553246e+00  3.85309243e+00  3.89523625e+00
    3.92482257e+00  3.93426943e+00  3.91735435e+00  3.87062383e+00
    3.79361176e+00  3.68817258e+00  3.55772305e+00  3.40620756e+00
    3.23572874e+00  3.05097556e+00  2.88121748e+00  2.76484323e+00
    2.70738125e+00  2.68866944e+00  2.68358946e+00  2.66993380e+00
    2.65009928e+00  2.63028550e+00  2.60823011e+00  2.58082247e+00
    2.54706120e+00  2.51035118e+00  2.53496933e+00  2.69541931e+00
    2.87949491e+00  2.86410069e+00  2.81640053e+00  2.80872631

In [25]:
print(output.cpu().numpy())

[[[[ 2.23377562e+00  2.74124503e+00  2.73276925e+00  2.75732708e+00
     2.78124714e+00  2.81436181e+00  2.86101437e+00  2.91756463e+00
     2.95249915e+00  2.98855090e+00  2.99776006e+00  3.02717972e+00
     3.04978561e+00  3.08638978e+00  3.08290386e+00  3.02671790e+00
     2.95475674e+00  2.91777945e+00  2.91627932e+00  2.91969109e+00
     2.91758490e+00  2.91471457e+00  2.91231155e+00  2.91575027e+00
     2.93043113e+00  2.96117353e+00  2.99776649e+00  3.03435326e+00
     3.06775427e+00  3.10126591e+00  3.13406825e+00  3.16202164e+00
     3.17931914e+00  3.18926525e+00  3.18802285e+00  3.15929818e+00
     3.10799432e+00  3.05692387e+00  3.00939488e+00  2.94968176e+00
     2.87379217e+00  2.79948521e+00  2.72576404e+00  2.64978242e+00
     2.56064558e+00  2.46633720e+00  2.37612414e+00  2.31770587e+00
     2.32891774e+00  2.40197611e+00  2.45805192e+00  2.45978332e+00
     2.44598532e+00  2.42696428e+00  2.41847658e+00  2.41531992e+00
     2.42824364e+00  2.43723154e+00  2.46367502e

## 5.转换格式(label转为最终的weights)

In [9]:
#将label面心值转为a值
def change_Label_to_a(all_vertical_edge_centers,all_horizontal_edge_centers):
    a_vertical = np.zeros((64, 65, 2))
    # a_vertical = np.random((64, 65, 2))
    a_horizontal = np.zeros((64, 65, 2))
    # a_horizontal = np.random.random((64, 65, 2))


    # 21. 求a:   横着的边，分两种情况，边缘（对称的）和非边缘的边.这里matrix_64要行列互换，因为横着时面心值是一列一列求得，竖着时是一行一行求的。
    for i in range(64):
        for j in range(65):
            if j == 0 or j == 64:
                # a_horizontal[i, j] = (all_horizontal_edge_centers[i, 0] - matrix_64[i, 0]) / (matrix_64[i, 63] - matrix_64[i, 0])#换之前
                a_horizontal[i, j] = (all_horizontal_edge_centers[i, 0] - matrix_64[0, i]) / (
                    matrix_64[63, i] - matrix_64[0, i])
            else:
                # aA=(1-a)B=E   a = (E-B)/(A-B) 其中：A为matrix_64[i,j]，B为matrix_64[i,j+1]
                # a_horizontal[i, j] = (all_horizontal_edge_centers[i, j] - matrix_64[i, j]) / (matrix_64[i, j-1] - matrix_64[i, j])
                a_horizontal[i, j] = (all_horizontal_edge_centers[i, j] - matrix_64[j, i]) / (
                    matrix_64[j - 1, i] - matrix_64[j, i])

    # 22. 求a:   竖着的边，分两种情况，边缘（对称的）和非边缘的边
    for i in range(64):
        for j in range(65):
            if j == 0 or j == 64:
                a_vertical[i, j] = (all_vertical_edge_centers[i, 0] - matrix_64[i, 0]) / (
                    matrix_64[i, 63] - matrix_64[i, 0])
            else:
                # aA=(1-a)B=E   a = (E-B)/(A-B) 其中：A为matrix_64[i,j]，B为matrix_64[i,j+1]
                # 2.错误matrix_64[i, j-1]) / (改成matrix_64[i, j]) / (
                a_vertical[i, j] = (all_vertical_edge_centers[i, j] - matrix_64[i, j]) / (
                    matrix_64[i, j - 1] - matrix_64[i, j])

    # # 若最终a对应的矩阵里面出现无穷，则将其替换为0.5.解决了分母为0的问题
    # a_vertical[np.isinf(a_vertical)] = 0.5
    # a_horizontal[np.isinf(a_horizontal)] = 0.5
    #这里64x65x2截成64x64x2,因为边框对称时值相同
    
    a_vertical = torch.tensor(a_vertical[:,:64,:])
    a_horizontal = torch.tensor(a_horizontal[:,:64,:])

    print(a_vertical.shape)
    print(a_horizontal.shape)
    
    return  a_vertical,a_horizontal

In [10]:
#返回一个列表，里面嵌套两个子列表，第一个子列表存放的是内部的a值，第二个子列表存放的是边框的a值，
#且顺序为上（左到右），下（左到右），左（下到上），右（下到上）
def conversion_format(output):
    # 调整output的维度顺序
    output = output.permute(0,2,3,1)  #torch.Size([1, 64, 64, 4])
    print(output.shape)
    output = output[0]#torch.Size([64, 64, 4])
    print(output.shape)
    # print(output)

    #将输出output拆成两个面心值
    all_vertical_edge_centers = output[:, :, 0:2].cpu()
    all_horizontal_edge_centers = output[:, :, 2:4].cpu()
#     print(all_vertical_edge_centers.shape)
#     print(all_horizontal_edge_centers.shape)

    
    a_vertical,a_horizontal  = change_Label_to_a(all_vertical_edge_centers,all_horizontal_edge_centers)    
    
    # 将前两个元素相加除以二得到一个元素(x+y/)2
    avg_vertical = (a_vertical[:, :, 0] + a_vertical[:, :, 1]) / 2
    # 将后两个元素相加除以二得到另一个元素
    avg_horizontal = (a_horizontal[:, :, 0] + a_horizontal[:, :, 1]) / 2

    # 重新组合成新的形状为(64, 64, 2)的张量
    new_avg_a_output = torch.stack([avg_vertical, avg_horizontal], dim=2)
    # 打印转换后的数据形状
#     print(new_avg_a_output.shape)

    #返回两个求完平均的面心值,包括两个64*64矩阵，矩阵是求完平均后的a值，一个竖着的，一个横着的
    vertical_1d = new_avg_a_output[:, :, 0]
    horizontal_1d = new_avg_a_output[:, :,1]
#     print(vertical_1d.shape)
#     print(horizontal_1d.shape)
    
    
    border = []#存所有边框，四个边框
    left_border=[]#存左边框
    bottom_border = []#存下边框
    inner = []#存内部的面心值

    #下面将2个64x64面心值变换格式，返回指定的格式result
    for i in range(len(vertical_1d)): #两个for循环等价于for i in range(64):
        for j in range(len(vertical_1d[i])):
            if j ==0:#j=0添加边框
                #添加左边框
                left_border.append(vertical_1d[i][0])
                #添加下边框
                bottom_border.append(horizontal_1d[i][0])
            else: 
                if i !=  63:#当竖着的最后一行时，上面没有对应的横着的
                    inner.append(vertical_1d[i][j])#竖着的
    #                 print("{j-1},{i+1}",j-1,i+1)
                    inner.append(horizontal_1d[j-1][i+1])   #再横着的   
                    if j == 63:#如果j=63的话，还需要再加入最后一列的横着的边
                        inner.append(horizontal_1d[63][i+1])#当i=63,横着的加最后一列 
                else:#if i ==63 :
                    inner.append(vertical_1d[63][j])#当i=63,inner最后添加竖着的一行竖线

    inner = [tensor.cpu().numpy().tolist() for tensor in inner]#将一维列表里面的tensor元素转为numpy格式，并返回cpu版本
    # print(inner)
    four_border = [bottom_border,bottom_border,left_border,left_border]#顺序是上（左到右），下（左到右），左（下到上），右（下到上）
    border = [item.numpy().tolist() for sublist in four_border for item in sublist]
    result = [inner,border]
    print(len(result))
    return result,a_vertical,a_horizontal,all_vertical_edge_centers,all_horizontal_edge_centers



In [11]:
# result  = conversion_format(output)
result,a_vertical,a_horizontal,all_vertical_edge_centers,all_horizontal_edge_centers  = conversion_format(output)

torch.Size([1, 64, 64, 4])
torch.Size([64, 64, 4])
torch.Size([64, 64, 2])
torch.Size([64, 64, 2])
2


In [12]:
print(a_vertical.numpy())


[[[-3.41746254e+01 -3.21236038e+00]
  [-1.94170494e+01 -6.02107882e-01]
  [ 1.25555162e+01 -5.13360882e-03]
  [ 4.23643303e+00  1.57134995e-01]
  [ 3.44578433e+00  3.47363681e-01]
  [ 5.69404793e+00  3.58033061e-01]
  [ 1.66535721e+01  4.15774405e-01]
  [ 2.50838490e+01  4.91233826e-01]
  [ 1.10429588e+02  2.59348333e-01]
  [-1.07161093e+01 -4.52213846e-02]
  [-4.31150007e+00 -4.17822868e-01]
  [-2.57189584e+00 -5.15474200e-01]
  [-1.66859925e+00 -4.50317621e-01]
  [-1.03318799e+00 -9.00351033e-02]
  [-5.02894282e-01  1.11730538e-01]
  [-4.06005085e-02 -4.45794225e-01]
  [ 3.97969037e-01  3.11027927e+01]
  [ 6.92693412e-01 -2.15224171e+01]
  [ 2.97358871e+00 -1.25766921e+00]
  [ 2.61806345e+00  1.30323768e-01]
  [ 5.63248873e+00  8.98557425e-01]
  [ 2.17289295e+01  1.84849250e+00]
  [ 6.42196274e+01  3.09076309e+00]
  [ 2.09041157e+01  2.63620591e+00]
  [ 7.09266901e+00  1.55038714e+00]
  [ 1.28521585e+01  4.50215960e+00]
  [-2.44935265e+01 -3.19417506e-01]
  [ 1.60995129e+03  6.194901

  [ 7.07989960e+01 -5.58622408e+00]]]


In [13]:
# print(a_horizontal.numpy())

In [14]:
# all_vertical_edge_centers.numpy()

In [15]:
print(len(result[0]))
print(len(result[1]))
print(8064+64*2)

8064
256
8192


In [16]:
result

[[-10.00957864522934,
  -9.821555614471436,
  6.275191317079589,
  1.5303094387054443,
  2.1967840120196342,
  9.01753881573677,
  1.896574005484581,
  1.7488510888069868,
  3.026040494441986,
  2.7279216647148132,
  8.534673243761063,
  3.251088500022888,
  12.787541389465332,
  4.0429262816905975,
  55.34446832537651,
  7.104808062314987,
  -5.380665330216289,
  -57.884800642728806,
  -2.36466147005558,
  -2.398252159357071,
  -1.5436850190162659,
  0.03194919228553772,
  -1.0594584345817566,
  0.9062215387821198,
  -0.5616115443408489,
  1.6334773898124695,
  -0.19558187201619148,
  2.753142476081848,
  -0.24319736659526825,
  2.3569278717041016,
  15.750380888581276,
  6.544109344482422,
  -10.41486182808876,
  11.482566118240356,
  0.8579597473144531,
  -160.67394924163818,
  1.3741936087608337,
  -53.453163146972656,
  3.265523076057434,
  -17.897831916809082,
  11.78871101140976,
  -19.55606746673584,
  33.655195236206055,
  -42.55203723907471,
  11.770160794258118,
  -69.041148

In [17]:
count = sum(1 for num in result[0] if 0 < num < 1)
print(count)
print(len(result[0]))
print(count/len(result[0]))

684
8064
0.08482142857142858


In [18]:
# train_loss,test_loss = np.load('./data/lossa/loss_model_500_optuna_02.npy')
# print(train_loss)
# print(test_loss)

In [19]:
# import matplotlib.pyplot as plt
# # 创建 x 轴数据，假设为 epoch 数
# epoch = np.arange(1, len(train_loss) + 1)

# train_loss = [round(int(a)/(all_data.shape[0]*0.8), 5) for a in train_loss]
# print(train_loss)
# # 绘制训练损失曲线
# plt.plot(epoch, train_loss, 'b', label='Train Loss')

# # 关闭科学计数法
# plt.gca().get_yaxis().get_major_formatter().set_scientific(False)

# # 设置y轴范围下限为0
# # plt.ylim(bottom=0)

# # 设置图表标题和轴标签
# plt.title('Train Loss')
# plt.xlabel('epoch')
# plt.ylabel('Loss')

# # 添加图例
# plt.legend()

# # 显示图表
# plt.show()

In [20]:
# # 创建 x 轴数据，假设为 epoch 数
# epoch = np.arange(1, len(test_loss) + 1)
# test_loss = [round(int(a)/(all_data.shape[0]*0.2), 5) for a in test_loss]
# print(test_loss)


# # 绘制测试损失曲线
# plt.plot(epoch, test_loss, 'r', label='Test Loss')

# # 设置图表标题和轴标签
# plt.title('Test Loss')
# plt.xlabel('epoch')
# plt.ylabel('Loss')

# # 添加图例
# plt.legend()

# # 显示图表
# plt.show()